In [54]:
import os
import numpy as np
import pandas as pd
import librosa
import IPython

In [56]:
path = 'dataset/' 

In [161]:
class Preprocessing:
    
    def __init__(self, data_path):
        self.path = path
        self.mp3_name = os.listdir(self.path)
        self.target = pd.DataFrame(columns=[['species', 'bird']], index=range(0,2))
        self.audio = np.zeros((len(self.mp3_name), 441000,), dtype=np.float32())
        
    def create_input_data(self):
        #converts .mp3 to float32 values and returns a numpy array
        OFFSET = 10.0
        DURATION = 20.0
        
        for i, name in enumerate(self.mp3_name):
            audio_dir = os.path.join(self.path, name)
            #librosa requires ffmpeg to load .mp3 files
            audio_data, sr = librosa.load(audio_dir, offset=OFFSET, duration=DURATION)
            self.audio[i] = audio_data
            
        return self.audio
        
    def create_target_data(self):
        #creates a pandas dataframe with two targets columns 
        #                                     -species & bird name
        species = []
        bird = []
        for i, name in enumerate(self.mp3_name):
            species.append(name.split('-')[0].lower())
            bird.append(name.split('-')[1])
        
        self.target[['species']] = np.array(species).reshape(-1, 1)
        self.target[['bird']] = np.array(bird).reshape(-1, 1)
        
        return (self.target)

In [156]:
preprocessing = Preprocessing(path)
data_x = preprocessing.create_input_data()
data_y = preprocessing.create_target_data()

In [157]:
data_x.shape

(2, 441000)

In [158]:
data_x

array([[-2.7295402e-05,  7.9479686e-04,  1.9936522e-03, ...,
         4.9520028e-03,  9.4234105e-03,  8.3365282e-03],
       [-2.9826986e-03, -5.0083739e-03, -5.5189175e-03, ...,
         3.9538797e-03,  3.7536446e-03,  4.5483671e-03]], dtype=float32)

In [159]:
data_y.shape

(2, 2)

In [160]:
data_y

,species,bird
0,acrocephalus,arundinaceus
1,acrocephalus,arundinaceus
